In [1]:
%pip install --upgrade pip
# install dependencies
%pip install -q amplpy matplotlib pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Select solver

SOLVER = "cplex"

from amplpy import AMPL, ampl_notebook, DataFrame

ampl = ampl_notebook(
    modules=["cplex"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Licensed to AMPL Community Edition License for <robert.cowlishaw.2017@uni.strath.ac.uk>.


In [3]:
import numpy as np
import csv
import os
import pandas as pd

In [4]:
# Load time dataset
SAVE_DIR = "data_icsmd_1day"

save_location = os.path.join(os.getcwd(), "..", "DATA", SAVE_DIR)
if not os.path.exists(save_location):
    raise Exception("Sat Data not setup, run get_active_tle.py first")

timer_all = np.load(save_location+"/all_combs_flat.npy")
timer_all[np.isnan(timer_all)] = 0

timer_all = np.array(timer_all, dtype=int)

timer = timer_all[:,0:5]

print(np.shape(timer))


(100, 5)


In [35]:
%%writefile OBJ.mod

# param N;

# set dim1 = {10, 10, 5};
# # param dim2;

# set sats = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11};
# set T {sats, sats, 1..5};

var x0 integer >= 0 <= 82;
# var x1 integer >= 0 <= N;
# var x2 integer >= 0 <= N;
# var x3 integer >= 0 <= N;

param f;

param a {1..7};

data;
param a := 1 10  2 15  3 24  4 12  5 15  6 12  7 19 ;


# Sorting
model;
set S ordered by Reals;
let S := setof {i in 1..7} a[i] + Uniform(0,0.0001);
let {i in 1..7} a[i] := floor(member(i,S));

display a;

set NUTR;
set FOOD;

minimize total_time: x0;

# s.t. copper: 4.04 * x1 + 2.02 * x2 <= 4800;

Overwriting OBJ.mod


In [36]:
m = AMPL()
m.read("OBJ.mod")


a [*] :=
1  10
2  12
3  12
4  15
5  15
6  19
7  24
;



In [38]:
# foods[food] = (cost, f_min, f_max)
foods = {
    "BEEF": (3.59, 2, 10),
    "CHK": (2.59, 2, 10),
    "FISH": (2.29, 2, 10),
    "HAM": (2.89, 2, 10),
    "MCH": (1.89, 2, 10),
    "MTL": (1.99, 2, 10),
    "SPG": (1.99, 2, 10),
    "TUR": (2.49, 2, 10),
}
# nutrients[nutr] = (n_min, n_max)
nutrients = {
    "A": (700, 20000),
    "C": (700, 20000),
    "B1": (700, 20000),
    "B2": (700, 20000),
    "NA": (0, 50000),
    "CAL": (16000, 24000),
}
m.set["FOOD"] = list(foods.keys())
m.param["cost"] = {food: cost for food, (cost, _, _) in foods.items()}
m.param["f_min"] = {food: f_min for food, (_, f_min, _) in foods.items()}
m.param["f_max"] = {food: f_max for food, (_, _, f_max) in foods.items()}
m.set["NUTR"] = list(nutrients.keys())
m.param["n_min"] = {nutr: n_min for nutr, (n_min, _) in nutrients.items()}
m.param["n_max"] = {nutr: n_max for nutr, (_, n_max) in nutrients.items()}
amounts = [
    [60, 8, 8, 40, 15, 70, 25, 60],
    [20, 0, 10, 40, 35, 30, 50, 20],
    [10, 20, 15, 35, 15, 15, 25, 15],
    [15, 20, 10, 10, 15, 15, 15, 10],
    [928, 2180, 945, 278, 1182, 896, 1329, 1397],
    [295, 770, 440, 430, 315, 400, 379, 450],
]
m.param["amt"] = {
    (nutrient, food): amounts[i][j]
    for i, nutrient in enumerate(nutrients)
    for j, food in enumerate(foods)
}

KeyError: 'An entity called cost cannot be found.'

In [33]:

timer = np.asanyarray(timer, dtype=int)
t_shape = np.shape(timer)

m.option["solver"] = SOLVER

m.param["f"] = 1

row_names = []
for i in range(10):
    for j in range(10):
        row_names.append(str(i+1)+"a"+str(j+1))
column_names = []
for i in range(5):
    column_names.append(str(i))

df_gd = pd.DataFrame(columns=column_names, index=row_names, data=timer)
print(df_gd)
for row in row_names:
    temp = []
    for col in column_names:
        temp.append(df_gd[col][row])
    # print(df_gd[column_names[0:2]][row])
    try:
        m.eval("set "+row+";")
    except:
        pass
    m.set[row] = set(temp)



m.eval("display a[2];")

print("done")

m.solve()

print(f'x = ({m.var["x0"].value():.1f})')
print(f'optimal value = {m.obj["total_time"].value():.2f}')

           0      1      2      3      4
1a1        0      0      0      0      0
1a2    64823      0      0      0      0
1a3        0      0      0      0      0
1a4        0      0      0      0      0
1a5    62212  62242      0      0      0
...      ...    ...    ...    ...    ...
10a6       0      0      0      0      0
10a7   59361  59391  59421  59451  59481
10a8       0      0      0      0      0
10a9       0      0      0      0      0
10a10      0      0      0      0      0

[100 rows x 5 columns]
Error:
	line 1 offset 4
	1a1 is already defined
	context:  set  >>> 1a1; <<< 
	
	For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>
	
Error:
	line 1 offset 4
	1a2 is already defined
	context:  set  >>> 1a2; <<< 
	
	For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>
	
Error:
	line 1 offset 4
	1a3 is already defined
	context:  set  >>> 1a3; <<< 
	
	For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl